In [14]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
client = OpenAI()

vector_store = client.beta.vector_stores.create(
  name="GRITAE2"
)
print(vector_store)

VectorStore(id='vs_0yhzHPvJzcQT66OJ23HtNfRp', created_at=1717766228, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=0, total=0), last_active_at=1717766228, metadata={}, name='GRITAE2', object='vector_store', status='completed', usage_bytes=0, expires_after=None, expires_at=None)


In [16]:
vector_store_files = client.beta.vector_stores.files.list(
  vector_store_id="vs_VwMZZPLE2n0cmHr0oqSgFzGL"
)
print(vector_store_files)

SyncCursorPage[VectorStoreFile](data=[VectorStoreFile(id='file-MUUy46T4HmjR1ylBCit5vpF5', created_at=1717765253, last_error=None, object='vector_store.file', status='completed', usage_bytes=653708, vector_store_id='vs_VwMZZPLE2n0cmHr0oqSgFzGL', chunking_strategy={'type': 'static', 'static': {'max_chunk_size_tokens': 800, 'chunk_overlap_tokens': 400}}), VectorStoreFile(id='file-MyyxS1VHkBQPQuJ6THPYH94S', created_at=1717765252, last_error=None, object='vector_store.file', status='completed', usage_bytes=720855, vector_store_id='vs_VwMZZPLE2n0cmHr0oqSgFzGL', chunking_strategy={'type': 'static', 'static': {'max_chunk_size_tokens': 800, 'chunk_overlap_tokens': 400}}), VectorStoreFile(id='file-DBWXLsIg8aNax7I7e8sfsq4l', created_at=1717765251, last_error=None, object='vector_store.file', status='completed', usage_bytes=692515, vector_store_id='vs_VwMZZPLE2n0cmHr0oqSgFzGL', chunking_strategy={'type': 'static', 'static': {'max_chunk_size_tokens': 800, 'chunk_overlap_tokens': 400}}), VectorStore

In [18]:
len(list(vector_store_files))

30

In [2]:
from openai import OpenAI
client = OpenAI()

my_assistant = client.beta.assistants.create(
    instructions="You have been given HTML documents of case studies of projects done for different companies in the past."
    " You will be provided the about page of a website of a company that deals in a specific field of IT servicing."
    " You are to provide the name of the company that corresponds to the most similar case study, "
    "and also 3 reasons why that case study is worth going through by the prospect that is, how it could be beneficial for them."
    " in json format only. " 
    "Example output: {\"url\":\"www.ABC.com\", \"points\":[\"We have helped ABC do X more efficiently which also could benefit you as you deal in similar fields\", \"We incorporated X into the ABC ecosystem... \", \" We helped ABC do this better, you could use it too.....\"]}",
    name="GRITAE",
    tools=[{"type": "file_search"}],
    model="gpt-4o", 
    tool_resources={
    "file_search": {
      "vector_store_ids": ["vs_VwMZZPLE2n0cmHr0oqSgFzGL"]
    },
    }
)
print(my_assistant)

Assistant(id='asst_O2HWobGq5XTAMxMJCt0bMiMT', created_at=1717830712, description=None, instructions='You have been given HTML documents of case studies of projects done for different companies in the past. You will be provided the about page of a website of a company that deals in a specific field of IT servicing. You are to provide the name of the company that corresponds to the most similar case study, and also 3 reasons why that case study is worth going through by the prospect that is, how it could be beneficial for them. in json format only. Example output: {"url":"www.ABC.com", "points":["We have helped ABC do X more efficiently which also could benefit you as you deal in similar fields", "We incorporated X into the ABC ecosystem... ", " We helped ABC do this better, you could use it too....."]}', metadata={}, model='gpt-4o', name='GRITAE', object='assistant', tools=[FileSearchTool(type='file_search')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_in

In [21]:
from openai import OpenAI
client = OpenAI()

my_assistant = client.beta.assistants.create(
    instructions="",
    name="GRITAE2",
    model="gpt-4o",
)
print(my_assistant)

Assistant(id='asst_EdCxX673x1nFzLPZhlD3yUZW', created_at=1717768602, description=None, instructions='', metadata={}, model='gpt-4o', name='GRITAE2', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)


In [22]:
thread = client.beta.threads.create()

In [ ]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id="asst_EdCxX673x1nFzLPZhlD3yUZW",
    model="gpt-4o",
    instructions="You will be given a list of endpoints of a particular website. You are to provide back the website link that is most likely to be the about page of the website. Return nothing but the link. The list is as follows: "+str()
)

In [11]:
import re


def extract_json_substring(text):
    # Define the regex pattern to match content between ```json and ```, excluding the markers
    pattern = r'```json\s*(.*?)\s*```'
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        # Extract and return the substring
        return match.group(1).strip()
    else:
        # Return None if no match is found
        return None

In [9]:
from openai import OpenAI
client = OpenAI()
thread = client.beta.threads.create()
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=my_assistant.id,
    model="gpt-4o",
    instructions="You have been given HTML documents of case studies of projects done for different companies in the past."
    "Provide the name of the company that corresponds to the most similar case study for the prospect www.loops.so, its details are provided in the files,"
    "and also 3 reasons why that case study is worth going through by the prospect that is, how it could be beneficial for them."
    " in json format only. " 
    "Example output: {\"case_study\":\"Fundrise\", \"points\":[\"We have helped ABC do X more efficiently which also could benefit you because...\", \"We incorporated X into the ABC ecosystem... \", \" We helped ABC do this better, you could use it too.....\"]}"
    )

In [12]:
import json
import re


if run.status == "completed":
        messages = client.beta.threads.messages.list(
            thread_id=thread.id,
            run_id=run.id
        )
        text = messages.data[0].content[0].text.value
        text = extract_json_substring(text)
        text = re.sub(r'【.*?】', '', text)
        text = text.replace("```json", "").replace("```","")
        text = json.loads(text)
        print(text)

{'case_study': 'Webex Events', 'points': ['Webex Events utilized Intercom to engage customers throughout their lifecycle, which could help Loops.so improve customer engagement and retention .', "They successfully transitioned from in-person to virtual events with Intercom's help during the pandemic, highlighting the platform's flexibility and scalability, which would be beneficial for Loops.so in adapting to market changes .", "By leveraging Intercom's Custom Bots, Webex Events improved lead qualification by 154%, which Loops.so could also achieve to enhance their sales pipeline efficiency ."]}
